In [1]:
from IPython.core.display import HTML, Javascript
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [1]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

> TODO: Überlegung, ob man die Positionen der Figuren auch anders für den Spieler angeben könnte
> Folge TOD: Funktionen zur Erstellung des Schachbretts abändern, aktualisieren
> TODO für neue Version: Benötigen die PIECE_LIST für die Erstellung der Schachbretter, da sie als Integer hinterlegt werden
> Überlegung für neue Version: Wir benötigen dieses Mal nicht nur uniques, sondern alles -> Beim Auslesen die "richtigen" Mengen erstellen?
> Weitere Überlegung: Wann sollte die Berechnung stattfinden (logischerweise abhängig von der Berechnungsdauer, aber dennoch eigentlich wichtige Frage)
> TODO für alle Funktionen: An neue Speicherart anpassen und anderen TODO's berücksichtigen

# Erstellung des vorgegebenen Bretts

In [1]:
WHITE_POSITIONS = {chess.KING:['e1'],
                   chess.QUEEN:['a6'],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:[]}

BLACK_POSITIONS = {chess.KING:['e7'],
                   chess.QUEEN:[],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:[]}

NameError: name 'chess' is not defined

> TODO: Funktionsbeschriftung
> TODO: Kann die Wahl der zu spielenden Situation nicht mit einem Dropdown umgesetzt werden?
> Anhand des Dropdowns kann daraufhin die Berechnung gestartet werden

In [ ]:
def get_occupied_cells():
    cells = []
    for values in WHITE_POSITIONS.values():
        for value in values:
            cells.append(value)
    for values in BLACK_POSITIONS.values():
        for value in values:
            cells.append(value)
    return cells

> TODO: Funktionsbeschriftung

In [ ]:
def check_for_correct_cells():
    cells = get_occupied_cells()
    for cell in cells:
        x = re.search("[a-h][1-8]", cell)
        if x is None:
            print(cell)
            print("Value incorrect!")
            return False
    return True

> TODO: Funktionsbeschriftung

In [ ]:
def collect_cells(color, pieces):
    occupied_cells = {}
    for piece_type, values in pieces:
            piece = chess.Piece(piece_type, color)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece
    return occupied_cells

> TODO: Funktionsbeschriftung

In [ ]:
def create_board():
    local_board = chess.Board(None)
    occupied_cells = {}
    if check_for_correct_cells():
        occupied_cells |= collect_cells(chess.WHITE, WHITE_POSITIONS.items())
        occupied_cells |= collect_cells(chess.BLACK, BLACK_POSITIONS.items())

        local_board.set_piece_map(occupied_cells)
        # Sollten wir nicht abhängig, ob ein Turm auf dem Spielfeld steht nicht hinterfragen, ob die Rochade bereits durchgeführt worden ist?
        local_board.castling_rights = local_board.clean_castling_rights()

        if not local_board.is_valid():
            display(local_board)
            print("Specified lineup is invalid")
            print("Default board created instead")
            local_board = chess.Board()
    return local_board

> TODO: Funktionsbeschriftung
> TODO: Abhängig von der Entscheidung wie gewählt wird, findet hier der Start der Berechnung statt bzw. das Laden der KI mit Piece List

In [ ]:
def load_ki():
    pass

# Neue Version von load_s_n_sequence(filename)

'''
def load_s_n_sequence(filename):
    s_n_sequence_tuples = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            for item in tmp:
                s_n_sequence_tuples.append(item)
    return s_n_sequence_tuples
'''

> TODO: Funktionsbeschriftung
> TODO: gilt es neu zu definieren, wobei es zu hinterfragen gibt, ob man wirklich die s_index + s_n_sequence als Übergabeparameter benötigt, weil wenn man durch die Menge geht kann man voraussichtlich schnell das nächste Brett finden
> TODO: find_situation_in_sequence muss ebenfalls überarbeitet werden, da nicht mehr in dieser Form notwendig

In [ ]:
def find_next_move():
    pass

# Globale Variablen für die Anzeige

> TODO: Wenn wirklich globale Variablen dann in Caps schreiben

In [ ]:
# Sollten diese nicht ebenfalls in caps geschrieben werden?
# TODO: Tooltip anpassen
input_field = widgets.Text()

execute_button = widgets.Button(
    description='Execute Move',
    disabled=False,
    button_style='',
    tooltip='Executes the written move.',
    icon='check'
)

# Globale Variablen für den aktuellen Zustand

> TODO: Sollte S_N_TUPLES einfach so definiert werden?
> TODO: Prüfen: Brauchen wir noch beide Indizes?
> TODO: Sollten nicht alle globalen Variablen an einem Ort definiert werden?

In [ ]:
S_N_TUPLES = load_s_n_sequence(FILE)
S_INDEX = 0
BOARD_INDEX = 0

> TODO: Funktionsbeschriftung
> TODO: Brauchen wir noch die Funktion?

In [ ]:
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

> TODO: Funktionsbeschriftung
> TODO: Muss FILENAME wirklich global sein?

In [ ]:
def update_filename(new_value):
    global FILENAME
    FILENAME = new_value

> TODO: Funktionsbeschriftung

In [ ]:
def reset_input_field():
    global input_field
    input_field.value = ''

> TODO: Funktionsbeschriftung

In [ ]:
def reset_board():
    global BOARD
    BOARD = create_board()
    global S_INDEX
    global BOARD_INDEX
    tupel = (BOARD.turn, BOARD.__str__())
    S_INDEX = find_situation_in_sequence(tupel, S_N_TUPLES)
    if BOARD.turn:
        S_INDEX, move = find_next_move(BOARD, S_INDEX, S_N_TUPLES)
        if S_INDEX == -1:
            print("No Move for white found")
        else:
            execute_move(move, BOARD.turn)
            reload_screen()

> TODO: Funktionsbeschriftung

In [ ]:
def execute_move(move, turn):
    global BOARD
    global S_INDEX
    global BOARD_INDEX
    move_file = open("Played_Games/" + FILENAME, "a")
    if turn:
        move_file.write(str(BOARD.fullmove_number) + ". " + move.uci() + " ")
    else:
        move_file.write(move.uci() + "\n")
    move_file.close()
    BOARD.push(move)
    tupel = (BOARD.turn, BOARD.__str__())
    S_INDEX = find_situation_in_sequence(tupel, S_N_TUPLES)

> TODO: Funktionsbeschriftung
> TODO: Überführung der beiden Dictionaries in ein Dictionary
> TODO: Brauchen wir die Überführung in Namen, reicht nicht das chess.SQUARE Objekt?

In [ ]:
def get_pieces_placed_on_board(color):
    piece_type_to_string = {
        1 : "Pawn",
        2 : "Knight",
        3 : "Bishop",
        4 : "Rook",
        5 : "Queen",
        6 : "King"
    }
    pieces_with_position = {
        "Pawn" : [],
        "Knight" : [],
        "Bishop" : [],
        "Rook" : [],
        "Queen" : [],
        "King" : []
    }
    for piece_square, color_piece in BOARD.piece_map().items():
        if color_piece.color == color:
            pieces_with_position[piece_type_to_string[color_piece.piece_type]].append(chess.square_name(piece_square))
    return pieces_with_position

> TODO: Funktionsbeschriftung
> TODO: Kann das square nicht ebenfalls ein chess.Square sein?

In [ ]:
def get_moves_from_square(square, legal_moves):
    moves = []
    for possible_move in legal_moves:
        if square == chess.square_name(possible_move.from_square):
            moves.append(possible_move)
    return moves

> TODO: Funktionsbeschriftung

In [ ]:
def reload_screen():
    clear_output()
    display(BOARD, input_field, execute_button)
    # Vorschlag: Diese Zeile in eine Funktion in eine Namens: Set_Focus setzen, damit man das nochmal sauber trennt, da ansonsten nicht ersichtlich weshalb diese Zeile Code notwendig ist
    display(Javascript("setTimeout(function focus() {document.querySelector('input').focus()}, 100);"))

> TODO: Funktionsbeschriftung

In [ ]:
def show_end_screen():
    clear_output()
    display(BOARD)

> TODO: Funktionsbeschriftung
> TODO: game_result so anpassen, dass nur noch die notwendigen Ergebnisse angezeigt werden können. Auch in Form eines Kommentars bemerken, weshalb wir uns dafür entschieden haben

In [ ]:
def game_result(result):
    if result == Termination.CHECKMATE:
        return get_color(BOARD.turn) + " has lost because of Checkmate!"
    elif result == Termination.STALEMATE:
        return "It's a draw!"
    elif result == Termination.INSUFFICIENT_MATERIAL:
        return "No side can win the game anymore!"
    elif result == Termination.SEVENTYFIVE_MOVES:
        return "The game is drawn because half-move clock is greater than 150 since a capture or a pwn has been moved."
    elif result == Termination.FIVEFOLD_REPETITION:
        return "The game is drawn because the current position occurred the fifth time!"
    elif result == Termination.FIFTY_MOVES:
        return "The game is drawn because half-move clock is greater than 100 since a capture or a pwn has been moved."
    elif result == Termination.THREEFOLD_REPETITION:
        return "The game is drawn because the current position occurred the third time!"
    elif result == Termination.VARIANT_WIN:
        return get_color(BOARD.turn) + " has won because of variant-specific conditions"
    elif result == Termination.VARIANT_LOSS:
        return get_color(BOARD.turn) + " has lost because of variant-specific conditions"
    elif result == Termination.VARIANT_DRAW:
        return "Game is drawn because of variant-specific conditions!"
    else:
        return "Something went wrong!"

> TODO: Funktionsbeschriftung
> TODO: Funktion in kleinere Unterfunktionen splitten
> TODO: Möglicherweise den Block: print, sleep, display

In [ ]:
def execute_entered_move(change):
    global BOARD
    global S_INDEX
    global BOARD_INDEX
    try:
        if input_field.value != '':
            black_move = chess.Move.from_uci(input_field.value)
            if black_move in list(BOARD.legal_moves):
                execute_move(black_move, BOARD.turn)
                reset_input_field()
                # Next AI move executed by white
                S_INDEX, next_move = find_next_move(BOARD, S_INDEX, S_N_TUPLES)
                if S_INDEX == -1:
                    print("No Move for white found")
                else:
                    execute_move(next_move, BOARD.turn)
                    reload_screen()
                if BOARD.legal_moves.count() == 0:
                    # If wanted add different endings
                    show_end_screen()
                    result = BOARD.outcome().termination
                    print(game_result(result))
            else:
                print("Entered a wrong move. Please try again!")
                print(input_field.value)
                time.sleep(2)
                reload_screen()
        else:
            print("Enter a move!")
            time.sleep(2)
            reload_screen()
    except ValueError:
        print("Entered a wrong move. Please try again!")
        print(input_field.value)
        time.sleep(2)
        reload_screen()

Binding der UI-Elemente

In [ ]:
execute_button.on_click(execute_entered_move)
input_field.on_submit(execute_entered_move)

> TODO: Funktionsbeschriftung
> TODO: Kürzerer Dateiname?

In [ ]:
def start_game():
    global BOARD
    BOARD = create_board()
    update_filename("Move-History_" + str(datetime.today().replace(microsecond=0)).replace(":","_") + ".txt")
    move_file = open("Played_Games/" + FILENAME, "a")
    move_file.write(BOARD.fen() + "\n")
    move_file.close()
    reset_board()